# Week 15 Programming Assignment

The final thing for this semester that we haven't exercised, yet, is working with databases through Python.  In thi final assignment of the semester, you will practice pulling data from either Google Big Query or Snowflake, loading that data into a Pandas data frame, summarizing the data, and then exporting that to an Excel file.

**You will need to use your own Google Big Query or Snowflake account to run this notebook, but you should try to make it configurable so anyone with an account in those technologies could run your notebook with minimal changes.**


Build a notebook with good comments (either in code or in markdown cells).  Then submit your assignment as usual.

### 1. Connect to the data source

Follow the examples provided in the [week15 folder of our GitHub repository](https://github.com/paulboal/hds5210-2023/tree/main/week15) to connect your notebook either to Big Query or Snowflake.

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
%load_ext google.colab.data_table

In [3]:
%%bigquery --project my-project-01-400323-407909
SELECT *
FROM my-project-01-400323-407909.Un_Insured.Population_uninsured_500_cities

Query is running:   0%|          |

Downloading:   0%|          |

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
0,2017,US,United States,None,US,BRFSS,Prevention,59,Current lack of health insurance among adults ...,%,...,15.5,None,None,308745538,None,PREVENT,ACCESS2,<NA>,<NA>,Health Insurance
1,2017,US,United States,None,US,BRFSS,Prevention,59,Current lack of health insurance among adults ...,%,...,15.0,None,None,308745538,None,PREVENT,ACCESS2,<NA>,<NA>,Health Insurance
2,2017,HI,Hawaii,Honolulu,City,BRFSS,Prevention,15003,Current lack of health insurance among adults ...,%,...,9.4,None,None,953207,"(21.4588039305, -157.973296737)",PREVENT,ACCESS2,15003,<NA>,Health Insurance
3,2017,HI,Hawaii,Honolulu,City,BRFSS,Prevention,15003,Current lack of health insurance among adults ...,%,...,9.2,None,None,953207,"(21.4588039305, -157.973296737)",PREVENT,ACCESS2,15003,<NA>,Health Insurance
4,2017,AL,Alabama,Birmingham,City,BRFSS,Prevention,0107000,Current lack of health insurance among adults ...,%,...,21.0,None,None,212237,"(33.5275663773, -86.7988174678)",PREVENT,ACCESS2,107000,<NA>,Health Insurance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28968,2017,WI,Wisconsin,Madison,Census Tract,BRFSS,Prevention,5548000-55025010600,Current lack of health insurance among adults ...,%,...,NaN,*,Estimates suppressed for population less than 50,1,"(43.0264053117, -89.3500269949)",PREVENT,ACCESS2,5548000,55025010600,Health Insurance
28969,2017,WI,Wisconsin,Milwaukee,Census Tract,BRFSS,Prevention,5553000-55079090200,Current lack of health insurance among adults ...,%,...,NaN,*,Estimates suppressed for population less than 50,17,"(43.0770733938, -88.0347229311)",PREVENT,ACCESS2,5553000,55079090200,Health Insurance
28970,2017,WI,Wisconsin,Milwaukee,Census Tract,BRFSS,Prevention,5553000-55079120300,Current lack of health insurance among adults ...,%,...,NaN,*,Estimates suppressed for population less than 50,9,"(42.9809580005, -87.9727193085)",PREVENT,ACCESS2,5553000,55079120300,Health Insurance
28971,2017,WI,Wisconsin,Racine,Census Tract,BRFSS,Prevention,5566000-55101001602,Current lack of health insurance among adults ...,%,...,NaN,*,Estimates suppressed for population less than 50,2,"(42.7727308725, -87.8565587843)",PREVENT,ACCESS2,5566000,55101001602,Health Insurance


Importing and authenticating:
1. Imported the module "google.colab" using the "import" statement. The "auth" module is imported to enable authentication with Google services.
2. The function auth.authenticate_user() verifies the identity of the user in order to grant them permission to utilize Google APIs.
3. The code "from google.colab.data_table import *" is used to import the necessary functionality for displaying tables in a visually appealing manner in Colab.


### 2. Query some data

Assuming the source database has some data in it or that you can load some data into it from any source, query it.  Then, read that data into a Pandas dataframe.

In [4]:
from google.cloud import bigquery
import pandas as pd

project_id = "my-project-01-400323-407909"
client = bigquery.Client(project=project_id)

sql_query = """
SELECT *
FROM my-project-01-400323-407909.Un_Insured.Population_uninsured_500_cities
"""

**SQL DATA QUERY**: This executes a BigQuery SQL query to retrieve all data from the Population_uninsured_500_cities table and presents the output in Colab.

### 3. Aggregate your data frame

Do some kind of aggregation on your data frame.  Something that makes sense and has some groups to it.  Don't just sum up one column for the entire data frame.  Be more creative than that.

In [5]:
df = client.query(sql_query).to_dataframe()

**Query converted to data frame**

In [6]:
df.drop_duplicates

<bound method DataFrame.drop_duplicates of        Year StateAbbr      StateDesc    CityName GeographicLevel DataSource  \
0      2017        US  United States        None              US      BRFSS   
1      2017        US  United States        None              US      BRFSS   
2      2017        HI         Hawaii    Honolulu            City      BRFSS   
3      2017        HI         Hawaii    Honolulu            City      BRFSS   
4      2017        AL        Alabama  Birmingham            City      BRFSS   
...     ...       ...            ...         ...             ...        ...   
28968  2017        WI      Wisconsin     Madison    Census Tract      BRFSS   
28969  2017        WI      Wisconsin   Milwaukee    Census Tract      BRFSS   
28970  2017        WI      Wisconsin   Milwaukee    Census Tract      BRFSS   
28971  2017        WI      Wisconsin      Racine    Census Tract      BRFSS   
28972  2017        WI      Wisconsin      Racine    Census Tract      BRFSS   

        

In [8]:
top_100_Uninsured_cities = (df.groupby(['CityName'])['Data_Value']
                      .agg('max')
                      .reset_index()
                      .sort_values(by='Data_Value', ascending=False)
                      .head(100))

In [9]:
top_100_Uninsured_cities = top_100_Uninsured_cities.rename(columns={'Data_Value' : '%_Uninsured'})

In [10]:
print
top_100_Uninsured_cities

,CityName,%_Uninsured
180,Houston,57.8
101,Dallas,57.5
53,Brownsville,56.1
367,San Antonio,55.8
122,El Paso,55.7
...,...,...
3,Albany,35.5
151,Gainesville,35.4
274,Nampa,35.4
322,Philadelphia,35.3


**Working with the data frame:**
1. I have imported the BigQuery data into a Pandas DataFrame for the purpose of analysis.
2. Next, I filtered the data to obtain only the percentage of the uninsured population. I accomplished this by using the "groupby" and "sort" functions.
Subsequently, I proceeded to rename the columns with more understandable titles before to downloading.


### 4. Write to Excel

Use Pandas functions to write your summarized data out to a local Excel file.

In [11]:
top_100_Uninsured_cities.to_excel('top_100_Uninsured.xlsx')

**EXPORTING THE DATA **
After I got the top uninsured cities data
I have downloaded and saved as a excel.

---

## Submitting Your Work

Submit your work as usual